In [1]:
from termcolor import colored
import ipywidgets as widgets
from datetime import date, datetime, timedelta
from traitlets import Unicode, validate
from scipy.signal import find_peaks
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

%run COVID_Info_functions.ipynb
%run Demographics_functions.ipynb
%run Plots_Functions.ipynb

In [2]:
def Get_Data(State_Name, ByPass_Update):
    State, State_Code = Get_State_Code(State_Name)
    # ###############################################################################################################
    print(colored('Obtaining Counties Geographical Data for the State of ' + State_Name, 'red', attrs=['bold']))
    Counties_Geo_df = Counties_Info(State_Code)
    print(colored('Obtaining Congressional Districts Geographical Data for the State of ' + State_Name, 
                  'red', attrs=['bold']))
    Districts_Geo_df = Congressional_Districts_Info(State_Code)
    # ###############################################################################################################
    print(colored('Obtaining Counties Demographics Data for the State of ' + State_Name, 'red', attrs=['bold']))
    Counties_Demo_df = Get_Counties_Demographics(State, State_Code)
    print(colored('Obtaining Congressional Districts Demographics Data for the State of ' + State_Name, 
                  'red', attrs=['bold']))
    Districts_Demo_df = Get_Districts_Demographics(State, State_Code)
    # ###############################################################################################################
    print(colored('Obtaining Counties COVID Data for the State of ' + State_Name, 'red', attrs=['bold']))
    if State == 'VA':
        Counties_COVID_df = Get_Cleaned_VA_COVID_data(ByPass_Update)
    else:
        print(colored('COVID data is only available for the state of Virginia right now', 
                      'blue', 'on_yellow', attrs=['bold']))
        return State_Code, [Counties_Geo_df, Districts_Geo_df, Counties_Demo_df, Districts_Demo_df]
    
    print(colored('Computing Congressional Districts COVID Data for the State of ' + State_Name, 
                  'red', attrs=['bold']))
    Districts_COVID_df = Get_CongressionalDistricts_COVID_data(State_Code, Counties_COVID_df)
    report_date_list = Districts_COVID_df['Report Date'].unique()
    
    Counties_COVID_df = Add_Pop_data(State_Code, 'County', Counties_COVID_df, 
                                     Counties_COVID_df.loc[:, 'Total Cases':'New Daily Deaths'].columns.to_list())
    Districts_COVID_df = Add_Pop_data(State_Code, 'Congressional District', Districts_COVID_df, 
                                      Districts_COVID_df.loc[:, 'Total Cases':'New Daily Deaths'].columns.to_list())
    # ###############################################################################################################
    print(colored('Obtaining Counties COVID Vaccination Data for the State of ' + State_Name, 'red', attrs=['bold']))
#     Counties_COVID_Vacc_df = Get_VA_COVID_Vaccines_data()
    Counties_COVID_Vacc_By_Sex_df = Get_VA_COVID_Vaccines_data_By_Sex()
    Counties_Vacc_Stat_Summary = Get_Counties_Vacc_Stat_Summary(State_Code, Counties_COVID_Vacc_By_Sex_df)
    
#     Counties_Vacc_Summary = Get_Counties_Vacc_Summary(State_Code, Counties_COVID_Vacc_df)
#     Counties_Vacc_Summary_Weekly = Get_Weekly_Vacc_Data(Counties_Vacc_Summary, 'County Code')

    print(colored('Computing Congressional Districts COVID Vaccination Data for the State of ' + State_Name, 
                  'red', attrs=['bold']))
    Districts_Vacc_Stat_Summary = Get_CongressionalDistricts_COVID_Vacc_Stat_data(State_Code, Counties_Vacc_Stat_Summary)
#     Districts_Vacc_Summary = Get_CongressionalDistricts_COVID_Vaccination_data(State_Code, Counties_Vacc_Summary)
#     Districts_Vacc_Summary_Weekly = Get_CongressionalDistricts_COVID_Vaccination_data(State_Code, 
#                                                                                       Counties_Vacc_Summary_Weekly)
    # ###############################################################################################################
    print(colored('Computing Monthly COVID Data for the State of ' + State_Name, 'red', attrs=['bold']))
    Counties_Monthly_COVID_df = Get_Monthly_Data(Counties_COVID_df, 'County Code')
    Districts_Monthly_COVID_df = Get_Monthly_Data(Districts_COVID_df, 'CDistrict')

    cols = ['Monthly Cases', 'Monthly Hospitalizations', 'Monthly Deaths']
    for col in cols:
        Counties_Monthly_COVID_df[col + '/10000 pop'] = (Get_Per_Pop(State_Code, 'County', 
                                                                     Counties_Monthly_COVID_df, col))*10000
        Districts_Monthly_COVID_df[col + '/10000 pop'] = (Get_Per_Pop(State_Code, 'Congressional District', 
                                                                     Districts_Monthly_COVID_df, col))*10000
    
    State_Monthly_COVID_df = Districts_Monthly_COVID_df.groupby('Date')[cols].sum().reset_index()
    
    State_Pop = Get_State_Pop19(State_Code)
    for col in cols:
        State_Monthly_COVID_df[col + '/10000 pop'] = (State_Monthly_COVID_df[col]/
                                                      State_Pop.loc[State_Code, 'Population'])*10000

    Month_order = CategoricalDtype(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], 
        ordered = True)
    
    State_Monthly_COVID_df['Date_ind'] = pd.to_datetime(State_Monthly_COVID_df['Date'], format = '%Y-%b')
    State_Monthly_COVID_df['Year'] = State_Monthly_COVID_df['Date_ind'].dt.year.astype(str)
    State_Monthly_COVID_df['Month'] = State_Monthly_COVID_df['Date_ind'].dt.month    
    d = dict(enumerate(calendar.month_abbr))
    State_Monthly_COVID_df['Month'] = (State_Monthly_COVID_df['Month'].map(d)).astype(Month_order)
    State_Monthly_COVID_df.set_index('Date_ind', inplace = True)
    State_Monthly_COVID_df.sort_index(inplace = True)
    
    Counties_Monthly_COVID_df['Date_ind'] = pd.to_datetime(Counties_Monthly_COVID_df['Date'], format = '%Y-%b')
    Counties_Monthly_COVID_df.Month = Counties_Monthly_COVID_df.Month.astype(Month_order)
    Counties_Monthly_COVID_df.reset_index(inplace = True)
    Counties_Monthly_COVID_df.set_index('Date_ind', inplace = True)
    Counties_Monthly_COVID_df.sort_index(inplace = True)
    Counties_Monthly_COVID_df['County Name'] = [(Get_CountyNames_Dict('51'))[x] for x in 
                                                Counties_Monthly_COVID_df['County Code']]
    
    Districts_Monthly_COVID_df['Date_ind'] = pd.to_datetime(Districts_Monthly_COVID_df['Date'], format = '%Y-%b')
    Districts_Monthly_COVID_df.Month = Districts_Monthly_COVID_df.Month.astype(Month_order)
    Districts_Monthly_COVID_df.reset_index(inplace = True)
    Districts_Monthly_COVID_df.set_index('Date_ind', inplace = True)
    Districts_Monthly_COVID_df.sort_index(inplace = True)
    # ###############################################################################################################
#     list_of_dfs = [Counties_Geo_df, Districts_Geo_df, Counties_Demo_df, Districts_Demo_df, 
#                    Counties_COVID_df, Districts_COVID_df, report_date_list, 
#                    Counties_Monthly_COVID_df, Districts_Monthly_COVID_df, State_Monthly_COVID_df, 
#                    Counties_Vacc_Summary, Districts_Vacc_Summary]
    list_of_dfs = [Counties_Geo_df, Districts_Geo_df, Counties_Demo_df, Districts_Demo_df, 
                   Counties_COVID_df, Districts_COVID_df, report_date_list, 
                   Counties_Monthly_COVID_df, Districts_Monthly_COVID_df, State_Monthly_COVID_df, 
                   Counties_Vacc_Stat_Summary, Districts_Vacc_Stat_Summary]
    
    return State_Code, list_of_dfs

In [3]:
def Get_Data_Map(State_Code, list_of_dfs):
    print(colored('Select options and click the Run button', 'red', 'on_yellow', attrs=['bold']))
    
    List_Keys_Dict = {('County', 'GEO'): 0, ('Congressional District', 'GEO'): 1, 
                      ('County', 'Partisan Voting Index'): 2, ('Congressional District', 'Partisan Voting Index'): 3,
                      ('County', 'Demographics, Age'): 2, ('Congressional District', 'Demographics, Age'): 3, 
                      ('County', 'Demographics, Race'): 2, ('Congressional District', 'Demographics, Race'): 3, 
                      ('County', 'Demographics, Income'): 2, ('Congressional District', 'Demographics, Income'): 3, 
                      ('County', 'COVID'): 4, ('Congressional District', 'COVID'): 5, 
                      ('None', 'None'): 7, ('County', 'None'): 7, ('Congressional District', 'None'): 7}
    
    if len(list_of_dfs) == 7:
        report_date_list = list_of_dfs[6]
    else:
        report_date_list = ['None']
        
    data_options = {'GEO': ['Population', 'Population Density', 'Land Area', 'Water Area'],
                    'COVID': ['Total Cases', 'Hospitalizations', 'Deaths', 
                              'New Daily Cases', 'New Daily Hospitalizations', 'New Daily Deaths', 
                              'Total Cases per Pop', 'Hospitalizations per Pop', 'Deaths per Pop', 
                              'New Daily Cases per Pop', 'New Daily Hospitalizations per Pop', 'New Daily Deaths per Pop', 
                              'Hospitalizations Ratio', 'Deaths Ratio', 'Deaths per Hospitalizations'], 
                    'Partisan Voting Index': ['PVI_R', 'PVI_D'],
                    'Demographics, Age': ['Total Population', 'Under 5 years', '5 to 14 years', '15 to 24 years', 
                                          'Under 18 years', '25 to 34 years', 'Under 35 years', '35 to 44 years', 
                                          '25 to 44 years', 'Under 45 years', '45 to 54 years', '55 to 64 years', 
                                          '45 to 64 years', '65 to 74 years',  '75 to 84 years', '65 years and over', 
                                          '85 years and over', 
                                          'Percent Under 5 years', 'Percent 5 to 14 years', 'Percent 15 to 24 years', 
                                          'Percent Under 18 years', 'Percent 25 to 34 years', 'Percent Under 35 years', 
                                          'Percent 35 to 44 years', 'Percent 25 to 44 years', 'Percent Under 45 years', 
                                          'Percent 45 to 54 years', 'Percent 55 to 64 years', 'Percent 45 to 64 years', 
                                          'Percent 65 to 74 years',  'Percent 75 to 84 years', 'Percent 65 years and over', 
                                          'Percent 85 years and over'], 
                    'Demographics, Race': ['White, Not Hispanic', 'Asian, Not Hispanic', 
                                           'Black or African American, Not Hispanic', 
                                           'American Indian & Alaska Native, Not Hispanic',
                                           'Native Hawaiin & Other Pacific Islander, Not Hispanic', 
                                           'Hispanic or Latino', 'Other, Not Hispanic', 'Two or More Races, Not Hispanic', 
                                           'Percent White, Not Hispanic', 'Percent Asian, Not Hispanic', 
                                           'Percent Black or African American, Not Hispanic', 
                                           'Percent American Indian & Alaska Native, Not Hispanic', 
                                           'Percent Native Hawaiin & Other Pacific Islander, Not Hispanic', 
                                           'Percent Hispanic or Latino', 'Percent Other, Not Hispanic', 
                                           'Percent Two or More Races, Not Hispanic'], 
                    'Demographics, Income': ['Median Income', 'Median Income, Male', 'Median Income, Female', 
                                             'Per Capita Income', 'Median Household Income', 'Median Income Female/Male']}
    
    date_options = {'GEO':['CD116'], 
                    'COVID':report_date_list[::-1], 
                    'Partisan Voting Index': ['2016, 2020 elections', '2012, 2016 elections'],
                    'Demographics, Age':['2019'], 
                    'Demographics, Race':['2019'], 
                    'Demographics, Income':['2019']}

    Log_Scale_W = widgets.Checkbox(value = False, description = 'Log Scale', indent = False)

    level_W = widgets.RadioButtons(options = ['County', 'Congressional District'], 
                                   description = 'Scope', 
                                   disabled = False)

    data_type_W = widgets.RadioButtons(options = data_options.keys(), 
                                       description = 'Data Type',  
                                       disabled=False)
    
    data_W = widgets.Dropdown(options = data_options[data_type_W.value])
    date_W = widgets.Dropdown(options = date_options[data_type_W.value])
    
    def update_data(*args):
        data_W.options = data_options[data_type_W.value]
        
    def update_date(*args):
        date_W.options = date_options[data_type_W.value]
        
    data_type_W.observe(update_data, 'value')
    data_type_W.observe(update_date, 'value')
    
    ##############################################################################################################
    def Map_State_Values(level, data_type, Log_Scale, date, data):
        k = List_Keys_Dict[(level, data_type)]
        data_col = data
        
        if k >= len(list_of_dfs):
            return
        
        if data_type == 'Partisan Voting Index':
            if k == 2:
                if (date == '2012, 2016 elections') or (State_Code != '51'):
                    print(colored('2016 PVI data for counties are not available', 'grey', 'on_yellow', attrs=['bold']))
                    return
            elif k == 3:
                if date == '2016, 2020 elections':
                    data_col = data.replace('PVI', 'PVI_2022')
                elif date == '2012, 2016 elections':
                    data_col = data.replace('PVI', 'PVI_2017')
        
        if Log_Scale and (data_type == 'Partisan Voting Index'):
            Log_Scale = False
            print(colored('Log_Scale reset to False. PVI data have negative values!', 'grey', 'on_yellow', attrs=['bold']))
        
        df = list_of_dfs[k].copy()
        
        if k == 4 or k == 5:
            mask = df['Report Date'] == date
            df = df[mask]
            if k == 4:
                df.set_index('County Code', inplace = True)
            elif k == 5:
                df.set_index('CDistrict', inplace = True)
        
        return Map_State(State_Code, level, df, data_col, Log_Scale)
    ###############################################################################################################
    
    widgets.AppLayout(left_sidebar = level_W, right_sidebar = data_type_W, footer = data_W)
    widget = widgets.interactive(Map_State_Values, {'manual' : True, 'manual_name' : 'Run'}, 
                                 level = level_W, data_type = data_type_W, Log_Scale = Log_Scale_W, 
                                 date = date_W, data = data_W)
    
    controls = widgets.HBox(widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
    output = widget.children[-1]
    display(widgets.VBox([controls, output]))

In [4]:
def Get_Animated_COVID_Map(State_Code, list_of_dfs):    
    report_date_list = list_of_dfs[-1]
    
    fmt = '%B %d %Y'
    
    report_dates = [(datetime.strptime(date, '%y/%m/%d').date()) for date in report_date_list]
    date_options = [(date.strftime(fmt), date) for date in report_dates]
    
    Log_Scale_W = widgets.Checkbox(value = False, description = 'Log Scale', indent = False)
    
    PerPop_W = widgets.Checkbox(value = False, description = 'Per Population', indent = False)

    scope_W = widgets.RadioButtons(options = ['Congressional District', 'County'],
                                   description = 'Scope',
                                   disabled = False)
    
    COVID_Data_W = widgets.RadioButtons(options = ['Cases', 'Hospitalizations', 'Deaths'], 
                                        description = 'Measure',
                                        disabled = False)
    
    Data_Count_W = widgets.RadioButtons(options = ['Total', 'Average'], 
                                        description = 'Count', 
                                        disabled = False)
    
    Dates_W = widgets.SelectionRangeSlider(options = date_options, 
                                           index = (0, len(date_options) - 1),
                                           layout = {'width': '700px', 'height': '80px'}, 
                                           style = {'description_width': '0px'},
                                           description = '',
                                           readout = False,
                                           disabled = False)
    
    Animated_Map_W = widgets.Checkbox(value = False, 
                                      description = 'Animated Map', 
                                      indent = False, 
                                      layout = {'width': '200px'})
    
    Plots_W = widgets.Checkbox(value = False, 
                               description = 'Time Series Plot', 
                               indent = False, 
                               layout = {'width': '200px'})
    
    Plot_Type_W = widgets.RadioButtons(options = ['scatter', 'line', 'bar'], 
                                       description = 'Plot Type', 
                                       disabled = False, 
                                       layout = {'visibility': 'hidden'})
    
    Normalized_W = widgets.Checkbox(value = False, 
                                    description = 'Normalized', 
                                    indent = False, 
                                    layout = {'visibility': 'hidden'})
    
    Add_State_W = widgets.Checkbox(value = False, 
                                   description = 'Compare to State-wide Values', 
                                   indent = False, 
                                   layout = {'visibility': 'hidden'})

    Wlist1 = ['From {}'.format((Dates_W.value[0]).strftime(fmt))]
    Wlist2 = ['From {}'.format(report_dates[0].strftime(fmt)), 
              '1 day', '7 days', '14 days', '21 days', '28 days']
    
    win_days_Dict = {'7 days': '7-day', '14 days': '14-day', '21 days': '21-day', '28 days': '28-day'}
    
    Window_W = widgets.Dropdown(options = Wlist1 + Wlist2, 
                                description = 'Window Size', 
                                layout = {'width': 'max-content'})
    
    SText_W = widgets.Label(value = 'Start Date:', layout = {'height': '20px'})
    EText_W = widgets.Label(value = 'End Date:', layout = {'height': '20px'})
    SLabel_W = widgets.Label(value = '{}'.format((Dates_W.value[0]).strftime(fmt)))
    ELabel_W = widgets.Label(value = '{}'.format((Dates_W.value[-1]).strftime(fmt)))
    
    freq_max = (Dates_W.value[-1] - Dates_W.value[0]).days
    freq_min = max(1, round(freq_max/120))

    Freq_W = widgets.BoundedIntText(
        value = freq_min,
        min = freq_min,
        max = freq_max,
        step = 1,
        description = 'Animation frequency in number of days', 
        style = {'description_width': '230px'}, 
        layout = {'height': '40px', 'visibility': 'hidden'},
#         layout={'width': '330px', 'height': '40px'},
        disabled = False
    )
    
    Nframes = 1 + round(((Dates_W.value[-1] - Dates_W.value[0]).days)/Freq_W.value)
    FLabel_W = widgets.Label(value = 'Number of Animation frames = {}'.format(Nframes), 
                             layout = {'visibility': 'hidden'})
    
    def update_Slabel(*args):
        SLabel_W.value = '{}'.format((Dates_W.value[0]).strftime(fmt))
        
    def update_Elabel(*args):
        ELabel_W.value = '{}'.format((Dates_W.value[-1]).strftime(fmt))
        
    def update_Window(*args):
        Wlist1 = ['From {}'.format((Dates_W.value[0]).strftime(fmt))]
        Window_W.options = Wlist1 + Wlist2
        
    def update_Freq(*args):
        if Dates_W.value[-1] <= Dates_W.value[0]:
            freq_max = 0
            freq_min = 0
        else:
            freq_max = (Dates_W.value[-1] - Dates_W.value[0]).days
            freq_min = max(1, round(freq_max/120))
            
        Freq_W.value = freq_min
        if freq_min > Freq_W.max:
            Freq_W.max = freq_max
            Freq_W.min = freq_min
        else:
            Freq_W.min = freq_min
            Freq_W.max = freq_max
            
        if Animated_Map_W.value:
            Freq_W.layout.visibility = 'visible'
        else:
            Freq_W.layout.visibility = 'hidden'
        
    def update_Flabel(*args):
        if Dates_W.value[-1] <= Dates_W.value[0]:
            Nframes = 1
        else:
            Nframes = 1 + round(((Dates_W.value[-1] - Dates_W.value[0]).days)/Freq_W.value)
        FLabel_W.value = "Number of Animation frames = {}".format(Nframes)
        
        if Animated_Map_W.value:
            FLabel_W.layout.visibility = 'visible'
        else:
            FLabel_W.layout.visibility = 'hidden'
            
    def update_Plot_Type(*args):
        if Plots_W.value:
            Plot_Type_W.layout.visibility = 'visible'
        else:
            Plot_Type_W.layout.visibility = 'hidden'
            
    def update_Normalized(*args):
        if Plots_W.value:
            Normalized_W.layout.visibility = 'visible'
        else:
            Normalized_W.layout.visibility = 'hidden' 
            
    def update_Add_State(*args):
        if Plots_W.value:
            Add_State_W.layout.visibility = 'visible'
        else:
            Add_State_W.layout.visibility = 'hidden'    
        
    Dates_W.observe(update_Slabel, 'value')
    Dates_W.observe(update_Elabel, 'value')
    Dates_W.observe(update_Window, 'value')
    Dates_W.observe(update_Freq, 'value')
    Dates_W.observe(update_Flabel, 'value')
    
    Animated_Map_W.observe(update_Freq, 'value')
    Animated_Map_W.observe(update_Flabel, 'value')
    
    Freq_W.observe(update_Flabel, 'value')
    
    Plots_W.observe(update_Plot_Type, 'value')
    Plots_W.observe(update_Normalized, 'value')
    Plots_W.observe(update_Add_State, 'value')
    
    ##############################################################################################################
    def Animated_Map_Values(scope, COVID_Data, Data_Count, Window, Per_Pop, Log_Scale, Dates, SLabel, ELabel, 
                            SText, EText, freq, FLabel, Animated_Map, Plots, Plot_Type, Normalized, Add_State):
        sdate = Dates[0]
        edate = Dates[-1]
    
        if edate <= sdate:
            Nframes = 0
            delta_dates = 0
        else:
            delta_dates = (edate - sdate).days
            Nframes = round((delta_dates)/freq)
            ffreq = ((edate - sdate).days)/Nframes
       
        if Nframes > 0:
            rdates = [(sdate + timedelta(days = int(i*ffreq))).strftime('%y/%m/%d') for i in range(Nframes + 1)]
            wdates = [(sdate + timedelta(days = i)).strftime('%y/%m/%d') for i in range(delta_dates + 1)]
        else:
            rdates = [edate.strftime('%y/%m/%d')]
            wdates = [edate.strftime('%y/%m/%d')]
        
        if scope == 'County':
            df = list_of_dfs[0].copy()
            col_list = ['Report Date', 'County Name', 'Total '+ COVID_Data, 'New Daily ' + COVID_Data]
            sIndex = 'County Code'
        elif scope == 'Congressional District':
            df = list_of_dfs[1].copy()
            col_list = ['Report Date', 'Total '+ COVID_Data, 'New Daily ' + COVID_Data]
            sIndex = 'CDistrict'

        df.set_index(sIndex, inplace = True)
        df.rename(columns={"Hospitalizations": "Total Hospitalizations", "Deaths": "Total Deaths"}, inplace = True)
        
        df.drop(df.columns.difference(col_list), axis = 1, inplace = True)
        mask = df['Report Date'] == report_date_list[0]
        df.loc[mask, 'New Daily ' + COVID_Data] = df.loc[mask, 'Total ' + COVID_Data]
        
        df_state = df.groupby('Report Date').sum()
        
        if Window == '1 day':
            New_Col = 'Daily ' + COVID_Data
            df[New_Col] = df['New ' + New_Col]
            df_state[New_Col] = df_state['New ' + New_Col]
        else:
            if Data_Count == 'Total':
                New_Col = COVID_Data
            elif Data_Count == 'Average':
                New_Col = 'Average ' + COVID_Data
            else:
                print('Error: Count value should be either \"Total" or \"Average" ')
                return
        
        if Window in Wlist2[2:]:
            win = int(Window.split()[0])
            if Data_Count == 'Total':
                df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                               rolling(win, min_periods = 1).sum().reset_index(1, drop = True).astype(int))
                df_state[New_Col] = (df_state['New Daily ' + COVID_Data].
                                     rolling(win, min_periods = 1).sum().astype(int))
            elif Data_Count == 'Average':
                df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                               rolling(win, min_periods = 1).mean().reset_index(1, drop = True).round(decimals = 2))
                df_state[New_Col] = (df_state['New Daily ' + COVID_Data].
                                     rolling(win, min_periods = 1).mean().round(decimals = 2))
        elif Window == Wlist2[0]:
            if Data_Count == 'Total':
                df[New_Col] = df['Total ' + COVID_Data]
                df_state[New_Col] = df_state['Total ' + COVID_Data]
            elif Data_Count == 'Average':
                df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                               expanding().mean().reset_index(1, drop = True).round(decimals = 2))
                df_state[New_Col] = df_state['New Daily ' + COVID_Data].expanding().mean().round(decimals = 2)
        
        df_all = df[df['Report Date'].isin(wdates)]
        df_state = df_state[df_state.index.get_level_values(0).isin(wdates)]
        
        df = df[df['Report Date'].isin(rdates)]
        
        if New_Col not in df:
            if Window == 'From {}'.format(sdate.strftime(fmt)):
                if Data_Count == 'Total':
                    df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                                   expanding().sum().reset_index(1, drop = True).astype(int))
                    df_all[New_Col] = (df_all.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                                       expanding().sum().reset_index(1, drop = True).astype(int))
                    df_state[New_Col] = (df_state['New Daily ' + COVID_Data].expanding().sum().astype(int))
                elif Data_Count == 'Average':
                    df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                                   expanding().mean().reset_index(1, drop = True).round(decimals = 2))
                    df_all[New_Col] = (df_all.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                                       expanding().mean().reset_index(1, drop = True).round(decimals = 2))
                    df_state[New_Col] = (df_state['New Daily ' + COVID_Data].expanding().mean().round(decimals = 2))                    
            else:
                print('Error in code: Where is the ', New_Col, ' column?!')
                return
            
        if Window.startswith('From'):
            win_date = Window.split(' ', 1)[1]
            title1 = 'Running ' + Data_Count
            title3 = ' since ' + win_date
        elif Window == '1 day':
            title1 = 'New Daily'
            title3 = ''
        else:
            if Data_Count == 'Total':
                title1 = 'Total Number of'
                title3 = ' in the last ' + Window
            elif Data_Count == 'Average':
                title1 = win_days_Dict[Window] + ' Rolling Average'
                title3 = ''
            
        if Per_Pop:
            data_col = New_Col + '/10000 pop'
            df[data_col] = (Get_Per_Pop(State_Code, scope, df, New_Col))*10000
            df_all[data_col] = (Get_Per_Pop(State_Code, scope, df_all, New_Col))*10000
            State_Pop = Get_State_Pop19(State_Code)
            df_state[data_col] = (df_state[New_Col]/State_Pop.loc[State_Code, 'Population'])*10000
            title2 = ' ' + COVID_Data + ' per 10,000 pop'
        else:
            data_col = New_Col
            title2 = ' ' + COVID_Data
            
        title = title1 + title2 + title3
        
        if Animated_Map:
            Map_Fig = Animated_Map_State(State_Code, scope, df, data_col, 'Report Date', title, Log_Scale)
        
        if Plots:
            if Normalized:
                df_all['Normalized ' + data_col] = (df_all.loc[:, data_col]/df_all.reset_index().
                                  groupby(sIndex)[data_col].max()).replace(np.inf, 0)
                df_state['Normalized ' + data_col] = (df_state.loc[:, data_col]/df_state[data_col].max()
                                                     ).replace(np.inf, 0)
                data_col = 'Normalized ' + data_col
                title = 'Normalized ' + title
                
            if Add_State:
                df_all.reset_index(inplace = True)
                df_all.set_index('Report Date', inplace = True)
                N_locations = len(df_all[sIndex].unique())
                df_all = pd.concat([df_state, df_all]).fillna('State-Wide')
                df_all.sort_index(inplace = True)
            
                if Normalized or data_col[-3:] == 'pop':
                    df_all[data_col + ' minus State-Wide value'] = (df_all.loc[:, data_col] - df_state.loc[:, data_col])
                    
                    df_all.reset_index(inplace = True)
                    df_all.sort_values(by = [sIndex, 'Report Date'], inplace = True)
                    df_all.set_index(sIndex, inplace = True)
                    
                    Plot_Fig = TimeSeries_Norm_Plots(State_Code, scope, Plot_Type, df_all, data_col, Log_Scale, title)
                else:
                    df_state_Av = df_state/N_locations
                    df_all = pd.concat([df_state_Av, df_all]).fillna('State Average')
                    df_all.sort_index(inplace = True)
                    df_all['Diff'] = (df_all.loc[:, data_col] - df_state_Av.loc[:, data_col])
                    
                    df_all['Ratio'] = (df_all.loc[:, 'Diff']/df_state.loc[:, data_col]).replace(np.inf, 0)
                    
                    df_all.reset_index(inplace = True)
                    df_all.sort_values(by = [sIndex, 'Report Date'], inplace = True)
                    df_all.set_index(sIndex, inplace = True)
                    
                    df_all.loc['State-Wide', 'Diff'] = np.nan
                    df_all.loc['State-Wide', 'Ratio'] = np.nan
                    
                    Plot_Fig = TimeSeries_Multi_Plots(State_Code, scope, Plot_Type, df_all, data_col, Log_Scale, title)
            else:
                Plot_Fig = TimeSeries_Plot(State_Code, scope, Plot_Type, df_all, data_col, Log_Scale, title)
                
        return
    ###############################################################################################################
    
    widget = widgets.interactive(Animated_Map_Values, {'manual' : True, 'manual_name' : 'Run'}, 
                                 scope = scope_W, COVID_Data = COVID_Data_W, Data_Count = Data_Count_W, 
                                 Window = Window_W, Per_Pop = PerPop_W, Log_Scale = Log_Scale_W, Dates = Dates_W, 
                                 SLabel = SLabel_W, ELabel = ELabel_W, SText = SText_W, EText = EText_W, 
                                 freq = Freq_W, FLabel = FLabel_W, Animated_Map = Animated_Map_W, Plots = Plots_W, 
                                 Plot_Type = Plot_Type_W, Normalized = Normalized_W, Add_State = Add_State_W)
    
    col1_1 = widgets.VBox([SText_W, SLabel_W])
    col1_2 = widgets.VBox([EText_W, ELabel_W])
    row1 = widgets.HBox(children=[col1_1, Dates_W, col1_2])

    col2 = widgets.VBox([PerPop_W, Log_Scale_W])
    row2 = widgets.HBox([scope_W, COVID_Data_W, col2, Data_Count_W, Window_W], 
                        layout = widgets.Layout(flex_flow = 'row wrap'))
    
    col3 = widgets.HBox([Freq_W, FLabel_W])
    row3 = widgets.HBox([Animated_Map_W, Freq_W, FLabel_W], 
                        layout = widgets.Layout(display = 'flex', 
                                                justify_content = 'space-between', 
                                                width = '90%'))
    
    col4 = widgets.VBox([Normalized_W, Add_State_W])
    
    row4 = widgets.HBox([Plots_W, Plot_Type_W, col4])
    
    output = widget.children[-1]
    display(widgets.VBox([row1, row2, row3, row4, widget.children[-2], output], 
                         layout = widgets.Layout(display="flex", justify_content="flex-start")))
    

In [5]:
from pandas.api.types import CategoricalDtype
def Get_Monthly_Plots(State_Code, list_of_dfs):
    df_county = list_of_dfs[0]
    df_district = list_of_dfs[1]
    df_state = list_of_dfs[2]
    #############################################################################################
    
    scope_options = {'State': 0, 'Congressional District': 1, 'County': 1}
    
    scope = widgets.RadioButtons(options = scope_options.keys(),
                                 description = 'Scope',
                                 disabled = False)
    
    X = widgets.RadioButtons(options = [('Date (Year/Month)', 'Date'), ('Month', 'Month')], 
                             description = 'X =', 
                             disabled = False)
    
    Y = widgets.RadioButtons(options = [('Cases', 'Monthly Cases'), 
                                        ('Hospitalizations', 'Monthly Hospitalizations'), 
                                        ('Deaths', 'Monthly Deaths')], 
                             description = 'Y = Monthly', 
                             disabled = False)
    
    PerPop = widgets.Checkbox(value = False, description = 'Per Population', indent = False)
    
    plot_options = {'Scatter': 0, 'Line': 1, 'Bar': 2, 'Box': 3}
    state_plot = widgets.RadioButtons(options = ['Scatter', 'Line', 'Bar'], 
                                      description = 'Plot Type', 
                                      disabled = False)
    county_plot = widgets.RadioButtons(options = plot_options.keys(), 
                                       description = 'Plot Type', 
                                       disabled = False)
    
    date_list = ['Month', 'Year']
    data_list = ['Monthly Cases', 'Monthly Hospitalizations', 'Monthly Deaths']
    
    scatter_color_s = widgets.Dropdown(options = [None] + date_list + data_list, 
                                       description = 'color', 
                                       layout = {'width': 'max-content', 'margin': '10px 0px 30px 0px'},
                                       disabled = False)
    
    scatter_size_s = widgets.Dropdown(options = [None] + data_list, 
                                      description = 'size', 
                                      layout = {'width': 'max-content', 'margin': '10px 0px 30px 10px'},
                                      disabled = False)
    
    scatter_symbol_s = widgets.Dropdown(options = [None] + date_list, 
                                        description = 'symbol', 
                                        layout = {'width': 'max-content', 'margin': '10px 0px 30px 50px'},
                                        disabled = False)
    
    Add_line_s = widgets.Checkbox(value = False, 
                                  description = 'Add Line',
                                  indent = False)
    
    row = widgets.HBox(children = [scatter_color_s, scatter_size_s, scatter_symbol_s])
    scatter_s = widgets.VBox(children = [row, Add_line_s])
    
    line_color_s = widgets.Dropdown(options = [None, 'Year'], 
                                    description = 'color', 
                                    layout = {'width': '180px'}, 
                                    disabled = False)
    
    line_dash_s = widgets.Dropdown(options = [None, 'Year'], 
                                   description = 'line-dash', 
                                   layout = {'width': '180px'}, 
                                   disabled = False)
    
    line_symbol_s = widgets.Dropdown(options = [None, 'Year'], 
                                     description = 'symbol', 
                                     layout = {'width': '180px'},
                                     disabled = False)
    
    line_s = widgets.HBox(children = [line_color_s, line_dash_s, line_symbol_s], 
                          layout = widgets.Layout(display = "flex", 
                                                  justify_content = "space-between", 
                                                  width = '80%'))
    
    bar_color_s = widgets.Dropdown(options = [None] + date_list + data_list, 
                                   description = 'color', 
                                   style = {'description_width': '100px'},
                                   disabled = False)
    
    bar_pattern_s = widgets.Dropdown(options = [None] + date_list, 
                                     description = 'pattern-shape', 
                                     style = {'description_width': '100px'},
                                     disabled = False)
    
    bar_s = widgets.VBox(children = [bar_color_s, bar_pattern_s])
    
    scatter_color_c = widgets.Dropdown(options = [scope.value] + date_list + data_list, 
                                       description = 'color', 
                                       layout = {'width': 'max-content', 'margin': '10px 0px 30px 0px'},
                                       disabled = False)
    
    scatter_size_c = widgets.Dropdown(options = [None] + data_list, 
                                      description = 'size', 
                                      layout = {'width': 'max-content', 'margin': '10px 0px 30px 10px'},
                                      disabled = False)
    
    scatter_symbol_c = widgets.Dropdown(options = [None, scope.value] + date_list, 
                                        description = 'symbol', 
                                        layout = {'width': 'max-content', 'margin': '10px 0px 30px 50px'},
                                        disabled = False)
    
    Add_line_c = widgets.Checkbox(value = False, 
                                  description = 'Add Line',
                                  indent = False)
    
    row = widgets.HBox(children = [scatter_color_c, scatter_size_c, scatter_symbol_c])
    scatter_c = widgets.VBox(children = [row, Add_line_c])
    
    line_color_c = widgets.Dropdown(options = [scope.value], 
                                    description = 'color', 
                                    layout = {'width': 'max-content'}, 
                                    disabled = False)
    
    if X.value == 'Date':
        line_dash_options = [None, 'Year', scope.value]
    elif X.value == 'Month':
        line_dash_options = ['Year']
    line_dash_c = widgets.Dropdown(options = line_dash_options, 
                                   description = 'line-dash', 
                                   layout = {'width': 'max-content'}, 
                                   disabled = False)
    
    line_symbol_c = widgets.Dropdown(options = [None, 'Year', scope.value], 
                                     description = 'symbol', 
                                     layout = {'width': 'max-content'},
                                     disabled = False)
    
    line_c = widgets.HBox(children = [line_color_c, line_dash_c, line_symbol_c], 
                          layout = widgets.Layout(display = "flex", 
                                                  justify_content = "space-between", 
                                                  width = '95%'))
    
    bar_color_c = widgets.Dropdown(options = [scope.value] + date_list + data_list, 
                                   description = 'color', 
                                   style = {'description_width': '100px'},
                                   disabled = False)
    
    bar_pattern_c = widgets.Dropdown(options = [None, scope.value, 'Year'], 
                                     description = 'pattern-shape', 
                                     style = {'description_width': '100px'},
                                     disabled = False)
    
    if bar_color_c.value != scope.value:
        bar_pattern_c.options = [scope.value]
    
    bar_c = widgets.VBox(children = [bar_color_c, bar_pattern_c])
    
    X_options = {'Date': [None, 'Year', 'Month'], 
                 'Month': [None, 'Year', 'Month', scope.value]}
    box_color_c = widgets.Dropdown(options = X_options[X.value], 
                                   description = 'color', 
                                   style = {'description_width': '100px'}, 
                                   disabled = False)
    box_c = widgets.HBox(children = [box_color_c])
    
    state_tabs = widgets.Tab(children = [scatter_s, line_s, bar_s])
    state_tabs.set_title(0, 'Scatter Plot')
    state_tabs.set_title(1, 'Line Plot')
    state_tabs.set_title(2, 'Bar Plot')
    state_tabs.selected_index = plot_options[state_plot.value]
    
    county_tabs = widgets.Tab(children = [scatter_c, line_c, bar_c, box_c])
    county_tabs.set_title(0, 'Scatter Plot')
    county_tabs.set_title(1, 'Line Plot')
    county_tabs.set_title(2, 'Bar Plot')
    county_tabs.set_title(3, 'Box Plot')
    county_tabs.selected_index = plot_options[county_plot.value]
    
    scope_tab1 = widgets.VBox(children = [state_plot, state_tabs])
    scope_tab2 = widgets.VBox(children = [county_plot, county_tabs])
    
    scope_tab = widgets.Tab(children = [scope_tab1, scope_tab2])
    scope_tab.set_title(0, 'State')
    scope_tab.set_title(1, 'County/District')
    scope_tab.selected_index = scope_options[scope.value]
    
    def update_scope_tab(*args):
        scope_tab.selected_index = scope_options[scope.value]
        
    def update_state_tab(*args):
        state_tabs.selected_index = plot_options[state_plot.value]
        
    def update_county_tab(*args):
        county_tabs.selected_index = plot_options[county_plot.value]
        
    def update_scatter_color_c(*args):
        scatter_color_c.options = [scope.value] + date_list + data_list
        
    def update_scatter_symbol_c(*args):
        scatter_symbol_c.options = [None, scope.value] + date_list
        
    def update_line_color_c(*args):
        line_color_c.options = [scope.value]
        
    def update_line_dash_c(*args):
        if X.value == 'Date':
            line_dash_options = [None, 'Year', scope.value]
        elif X.value == 'Month':
            line_dash_options = ['Year']
        line_dash_c.options = line_dash_options
        
    def update_line_symbol_c(*args):
        line_symbol_c.options = [None, 'Year', scope.value]
        
    def update_bar_color_c(*args):
        bar_color_c.options = [scope.value] + date_list + data_list
        
    def update_bar_pattern_c(*args):
        if bar_color_c.value != scope.value:
            bar_pattern_c.options = [scope.value]
        else:
            bar_pattern_c.options = [None, scope.value, 'Year']
            
    def update_box_color_c(*args):
        X_options = {'Date': [None, 'Year', 'Month'], 
                     'Month': [None, 'Year', 'Month', scope.value]}
        box_color_c.options = X_options[X.value]
    
    scope.observe(update_scope_tab, 'value')
    state_plot.observe(update_state_tab, 'value')
    county_plot.observe(update_county_tab, 'value')
    scope.observe(update_scatter_color_c, 'value')
    scope.observe(update_scatter_symbol_c, 'value')
    scope.observe(update_line_color_c, 'value')
    scope.observe(update_line_dash_c, 'value')
    X.observe(update_line_dash_c, 'value')
    scope.observe(update_line_symbol_c, 'value')
    scope.observe(update_bar_color_c, 'value')
    scope.observe(update_bar_pattern_c, 'value')
    bar_color_c.observe(update_bar_pattern_c, 'value')
    scope.observe(update_box_color_c, 'value')
    X.observe(update_box_color_c, 'value')

    # =============================================================================================================
    def Get_Plots(scope, x, y, PerPop, state_plot, county_plot, scatter_color_s, scatter_color_c, scatter_size_s, 
                  scatter_size_c, scatter_symbol_s, scatter_symbol_c, Add_line_s, Add_line_c, 
                  line_color_s, line_color_c, line_dash_s, line_dash_c, line_symbol_s, line_symbol_c, 
                  bar_color_s, bar_color_c, bar_pattern_s, bar_pattern_c, box_color_c):
        if scope == 'State':
            df = df_state.copy()
            plot_type = state_plot            
            if x == 'Month':
                df.sort_values(by = ['Month', 'Year'], inplace = True)
            if plot_type == 'Scatter':
                color = scatter_color_s
                size = scatter_size_s
                symbol = scatter_symbol_s
                addline = Add_line_s
                hover_info = ['Date']
            elif plot_type == 'Line':
                color = line_color_s
                line_dash = line_dash_s
                symbol = line_symbol_s
            elif plot_type == 'Bar':
                color = bar_color_s
                pattern = bar_pattern_s
                hover_info = ['Year']
                
            if PerPop:
                title = y + '/10000 pop in whole state'
            else:
                title = y + ' in whole state'
        else:
            scope_dict = {'Congressional District': 'CDistrict', 'County': 'County Name'}
            if scope == 'Congressional District':
                df = df_district.copy()
                hover_info = []
            elif scope == 'County':
                df = df_county.copy()
                hover_info = ['County Code']

            plot_type = county_plot            
            if x == 'Month':
                df.sort_values(by = ['Month', 'Year', scope_dict[scope]], inplace = True)
            elif x =='Date':
                df.sort_values(by = ['Date_ind', scope_dict[scope]], inplace = True)
                
            if plot_type == 'Scatter':
                color = scatter_color_c.replace(scope, scope_dict[scope])
                size = scatter_size_c
                if type(scatter_symbol_c) is str:
                    symbol = scatter_symbol_c.replace(scope, scope_dict[scope])
                else:
                    symbol = scatter_symbol_c
                
                addline = Add_line_c
                hover_info = hover_info + ['Date', scope_dict[scope]]
            elif plot_type == 'Line':
                color = line_color_c.replace(scope, scope_dict[scope])
                if line_dash_c == scope:
                    line_dash = line_dash_c.replace(scope, scope_dict[scope])
                else:
                    line_dash = line_dash_c
                if line_symbol_c == scope:
                    symbol = line_symbol_c.replace(scope, scope_dict[scope])
                else:
                    symbol = line_symbol_c
            elif plot_type == 'Bar':
                color = bar_color_c.replace(scope, scope_dict[scope])
                if bar_pattern_c == scope:
                    pattern = scope_dict[scope]
                else:
                    pattern = bar_pattern_c
                hover_info = hover_info + ['Year', scope_dict[scope]]
            elif plot_type == 'Box':
                if box_color_c == scope:
                    color = scope_dict[scope]
                else:
                    color = box_color_c
            
            if PerPop:
                title = y + '/10000 pop in each ' + scope
            else:
                title = y + ' in each ' + scope 
        
        if plot_type == 'Scatter':
            Fig = Scatter_Plot(df, scope, x, y, color, size, symbol, addline, hover_info, title, PerPop)
        elif plot_type == 'Line':
            Fig = Line_Plot(df, scope, x, y, color, line_dash, symbol, title, PerPop)
        elif plot_type == 'Bar':
            Fig = Bar_Plot(df, scope, x, y, color, pattern, hover_info, title, PerPop)
        elif plot_type == 'Box':
            Fig = Box_Plot(df, x, y, color, PerPop)
        
        return
    # ============================================================================================================
    
    widget = widgets.interactive(Get_Plots, {'manual' : True, 'manual_name' : 'Run'}, 
                                 scope = scope, x = X, y = Y, PerPop = PerPop, 
                                 state_plot = state_plot, county_plot = county_plot,
                                 scatter_color_s = scatter_color_s, scatter_color_c = scatter_color_c,
                                 scatter_size_s = scatter_size_s, scatter_size_c = scatter_size_c, 
                                 scatter_symbol_s = scatter_symbol_s, scatter_symbol_c = scatter_symbol_c, 
                                 Add_line_s = Add_line_s, Add_line_c = Add_line_c, 
                                 line_color_s = line_color_s, line_color_c = line_color_c, 
                                 line_dash_s = line_dash_s, line_dash_c = line_dash_c,
                                 line_symbol_s = line_symbol_s, line_symbol_c = line_symbol_c, 
                                 bar_color_s = bar_color_s, bar_color_c = bar_color_c, 
                                 bar_pattern_s = bar_pattern_s, bar_pattern_c = bar_pattern_c, 
                                 box_color_c = box_color_c)
    
    output = widget.children[-1]
    run_button = widget.children[-2]

    row1 = widgets.HBox([scope, X])
    row2 = widgets.HBox([Y, PerPop])
    
    display(widgets.VBox([row1, row2, scope_tab, run_button, output]))

    return

In [6]:
def Get_Animated_COVID_Vaccination_Map(State_Code, list_of_dfs):
    scope_W = widgets.RadioButtons(options = ['Congressional District', 'County'],
                                   description = 'Scope',
                                   disabled = False)
    
#     Vacc_Data_W = widgets.RadioButtons(options = ['people with at least 1 dose', 
#                                                   'people with at least 2 doses', 
#                                                   'people with at least 3 doses', 
#                                                   'people with at least 4 doses'], 
#                                        layout = {'width': '400px'},
#                                        description = 'Measure', 
#                                        disabled = False)
    
    Vacc_Data_W = widgets.RadioButtons(options = ['At Least One Dose', 
                                                  'Fully Vaccinated', 
                                                  'First Booster', 
                                                  'Second Booster', 
                                                  'Not Reported', 
                                                  'Immunocompromised Dose'], 
                                       layout = {'width': '400px'},
                                       description = 'Measure', 
                                       disabled = False)
    
    PerPop_W = widgets.Checkbox(value = False, 
                                description = 'Per Population', 
                                layout = {'width': '120px'},
                                indent = False)
    
    Animated_Map_W = widgets.Checkbox(value = False, 
                                      description = 'Animated Map', 
                                      indent = False, 
                                      layout = {'width': '180px'})
    
    Animation_Frequency_W = widgets.RadioButtons(options = ['Latest', 'Weekly', 'Biweekly', 'Monthly'], 
                                                 description = 'Animation Frequency', 
                                                 style = {'description_width': '200px'}, 
                                                 layout = {'width': '400px'},
                                                 disabled = False)
    
    Plots_W = widgets.Checkbox(value = False, 
                               description = 'Time Series Plot', 
                               indent = False, 
                               layout = {'width': '700px'})
    
#     ##############################################################################################################
    def Animated_Map_Values(scope, Vacc_Data, PerPop, Animated_Map, Animation_Frequency, Plots):
        if PerPop:
            data_col = Vacc_Data + ' per Pop'
            title = 'Percent of Vaccinated ' + Vacc_Data
        else:
            data_col = Vacc_Data
            title = 'Number of Vaccinated ' + Vacc_Data
            
        if scope == 'County':
            df = list_of_dfs[0].copy()
            sIndex = 'County Code'
            col_list = ['Date', sIndex, 'County Name', data_col]
        elif scope == 'Congressional District':
            df = list_of_dfs[1].copy()
            sIndex = 'CDistrict'
            col_list = ['Date', sIndex, data_col]

        df.drop(df.columns.difference(col_list), axis = 1, inplace = True)
        df_pivot = df.pivot(columns = sIndex)
        df_pivot = df_pivot.loc[(df_pivot != 0).any(axis = 1)]
        df = df_pivot.stack().reset_index()
        df.set_index('Date', inplace = True)
        
        if Animated_Map:
            if Animation_Frequency == 'Weekly':
                df_anime = Get_Weekly_Vacc_Data(df, sIndex, 'W-')
            elif Animation_Frequency == 'Biweekly':
                df_anime = Get_Weekly_Vacc_Data(df, sIndex, '2W-')
            elif Animation_Frequency == 'Monthly':
                start = df.index.min()
                end = df.index.max()
                rng = (pd.date_range(start.strftime('%Y-%m'), end.strftime('%Y-%m'), freq='MS') + 
                       pd.DateOffset(days = end.day - 1))
                if rng[0].strftime('%Y-%m-%d') < start.strftime('%Y-%m-%d'):
                    index_list = rng[1:]
                else:
                    index_list = rng
                df_anime = df.loc[index_list].copy()
                df_anime.index.names = ['Date']
            else:
                df_anime = df.loc[df.index.max()].copy()
            
            df_anime.reset_index(inplace = True)
            df_anime.set_index(sIndex, inplace = True)
            df_anime['Date'] =  pd.to_datetime(df_anime['Date']).dt.strftime('%y/%m/%d')
    
            Map_Fig = Animated_Map_State(State_Code, scope, df_anime, data_col, 'Date', title)
        
        if Plots:
            Plot_Fig = TimeSeries_VaccData_Plot(df, sIndex, data_col, title)
#             display(df)

#                 Plot_Fig = TimeSeries_Plot(State_Code, scope, Plot_Type, df_all, data_col, Log_Scale, title)
                
        return
    ###############################################################################################################
    
    widget = widgets.interactive(Animated_Map_Values, {'manual' : True, 'manual_name' : 'Run'}, 
                                 scope = scope_W, Vacc_Data = Vacc_Data_W, PerPop = PerPop_W, 
                                 Animated_Map = Animated_Map_W, Animation_Frequency = Animation_Frequency_W, 
                                 Plots = Plots_W)

    row1 = widgets.HBox([scope_W, Vacc_Data_W, PerPop_W])    
    row2 = widgets.HBox([Animated_Map_W, Animation_Frequency_W])
    row3 = widgets.HBox([Plots_W, widget.children[-2]])
    output = widget.children[-1]
    
    display(widgets.VBox([row1, row2, row3, output], 
                         layout = widgets.Layout(display = "flex", 
                                                 justify_content = "flex-start")))

    return
    

In [7]:
def get_trend(df_local, col):
    dataset = df_local[col].copy()
    dataset.index = pd.to_datetime(dataset.index, yearfirst = True)
    decomp = sm.tsa.seasonal_decompose(dataset, model = 'add', extrapolate_trend = 'freq')
    return decomp.trend

In [8]:
def Obtain_COVID_Waves_Info(list_of_dfs, show_plots = False):
    Waves_Info = {}
    
    df = list_of_dfs[0].copy()
    report_dates = list_of_dfs[1]
    COVID_Data_list = ['Cases', 'Hospitalizations', 'Deaths']
    sIndex = 'CDistrict'
    win = 60
    win2 = 7
    Wprom = 0.05
    Wdist = 90
    
    df.set_index(sIndex, inplace = True)
    df.rename(columns={"Hospitalizations": "Total Hospitalizations", "Deaths": "Total Deaths"}, inplace = True)
    col_list = ['Report Date', 'County Name']
    mask = df['Report Date'] == report_dates[0]
    
    for COVID_Data in COVID_Data_list:
        col_list.append('Total ' + COVID_Data)
        col_list.append('New Daily ' + COVID_Data)
        df.loc[mask, 'New Daily ' + COVID_Data] = df.loc[mask, 'Total ' + COVID_Data]
    
    df.drop(df.columns.difference(col_list), axis = 1, inplace = True)
    df_state = df.groupby('Report Date').sum()
    
    for COVID_Data in COVID_Data_list:
        New_Col = 'Total Number of ' + COVID_Data + ' in the last ' + str(win) + ' days'
        New_Col2 = 'Total Number of ' + COVID_Data + ' in the last ' + str(win2) + ' days'
        df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].rolling(win, min_periods = 1).sum().
                       reset_index(1, drop = True).astype(int))
        df_state[New_Col] = df_state['New Daily ' + COVID_Data].rolling(win, min_periods = 1).sum().astype(int)
        df[New_Col2] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].rolling(win2, min_periods = 1).sum().
                        reset_index(1, drop = True).astype(int))
        df_state[New_Col2] = df_state['New Daily ' + COVID_Data].rolling(win2, min_periods = 1).sum().astype(int)

        df['Normalized ' + New_Col] = (df.loc[:, New_Col]/(df.reset_index().groupby(sIndex)[New_Col].max())).replace(np.inf, 0)
        df_state['Normalized ' + New_Col] = (df_state.loc[:, New_Col]/df_state[New_Col].max()).replace(np.inf, 0)
        df['Normalized ' + New_Col2] = (df.loc[:, New_Col2]/(df.reset_index().groupby(sIndex)[New_Col2].max())
                                       ).replace(np.inf, 0)
        df_state['Normalized ' + New_Col2] = (df_state.loc[:, New_Col2]/df_state[New_Col2].max()).replace(np.inf, 0)
        
    DC_Ratio = df_state['Total Deaths'].iloc[-1]/df_state['Total Cases'].iloc[-1]
    DH_Ratio = df_state['Total Deaths'].iloc[-1]/df_state['Total Hospitalizations'].iloc[-1]
    
    new_col = 'Wave in the last ' + str(win) + ' days'
    c_col = 'Total Number of Cases in the last ' + str(win) + ' days'
    h_col = 'Total Number of Hospitalizations in the last ' + str(win) + ' days'
    d_col = 'Total Number of Deaths in the last ' + str(win) + ' days'
    df_state[new_col] = DC_Ratio*df_state[c_col] + DH_Ratio*df_state[h_col] + df_state[d_col]
    df_state['Normalized Wave'] = (df_state.loc[:, new_col]/df_state[new_col].max()).replace(np.inf, 0)
    
    Wave_Trend = get_trend(df_state, 'Normalized Wave')
    indices = find_peaks(Wave_Trend, prominence = Wprom, distance = Wdist)
    v_indices = find_peaks(-Wave_Trend, prominence = Wprom, distance = Wdist)
    
    valleys = v_indices[0]
    peaks = indices[0]
    st_valleys = [0]
    for i in valleys:
        st_valleys.append(i + 1)
    
    valleys_dates = pd.to_datetime(report_dates[valleys], yearfirst = True)
    peaks_dates = pd.to_datetime(report_dates[peaks], yearfirst = True)
    start_dates = pd.to_datetime(report_dates[st_valleys], yearfirst = True)
    
    for i in range(len(peaks)):
        key = 'Wave ' + str(i + 1)
        try:
            value = {'start': (st_valleys[i], start_dates[i]), 
                     'peak': (peaks[i], peaks_dates[i]), 
                     'end': (valleys[i], valleys_dates[i])}
        except IndexError:
            value = {'start': (st_valleys[i], start_dates[i]), 
                     'peak': (peaks[i], peaks_dates[i]), 
                     'end': ('NA', 'NA')}
        Waves_Info[key] = value
    
    if not show_plots:
        return Waves_Info
    
    title = 'Wave Trend with corresponding peaks and valleys'
    fig = px.line(Wave_Trend)
    trace = px.scatter(Wave_Trend.iloc[indices[0]], color_discrete_sequence = ['red'])["data"]
    fig.add_trace(trace[0])
    trace = px.scatter(Wave_Trend.iloc[valleys], color_discrete_sequence = ['green'])["data"]
    fig.add_trace(trace[0])
    
    fig.update_yaxes(title_text = 'Normalized Wave Trend Value')
    fig.update_traces(marker={'size': 15})
    fig.update_layout(title = title, title_x = 0.5,  title_font_color = 'royalblue', showlegend = False)
    fig.show(renderer = 'browser')
    
    # ===================================================================================
    def Get_dataset_trends(df_local, col_list):
        series_dict = {}
        for col in col_list:
            trend = get_trend(df_local, col)
            for COVID_Data in COVID_Data_list:
                if COVID_Data in col:
                    col_trend = COVID_Data + ' Trend'
                    series_dict[col_trend] = trend
                    break
            
        return pd.concat(series_dict, axis = 1)
    # ===================================================================================
    col_list = []
    col_list2 = []
    for COVID_Data in COVID_Data_list:
        col = 'Normalized Total Number of ' + COVID_Data + ' in the last ' + str(win) + ' days'
        col2 = 'Normalized Total Number of ' + COVID_Data + ' in the last ' + str(win2) + ' days'
        col_list.append(col)
        col_list2.append(col2)
    
    datasets_list = []
    datasets_list2 = []
    titles_list = []

    datasets_list.append(Get_dataset_trends(df_state, col_list))
    datasets_list2.append(Get_dataset_trends(df_state, col_list2))
    titles_list.append('State')
    
    Districts = list(df.index.unique())

    for i in range(len(Districts)):
        d = Districts[i]
        df_District = df.loc[d]
        df_District.set_index('Report Date', inplace = True)
        datasets_list.append(Get_dataset_trends(df_District, col_list))
        datasets_list2.append(Get_dataset_trends(df_District, col_list2))
        titles_list.append('District ' + d)

    Mytitle1 = 'Total Number of COVID Cases, Hospitalizations, & Deaths in the last ' + str(win) + ' days'
    Mytitle2 = 'Total Number of COVID Cases, Hospitalizations, & Deaths in the last ' + str(win2) + ' days'
    Trend_Plots(datasets_list, titles_list, Mytitle1, Waves_Info)
    Trend_Plots(datasets_list2, titles_list, Mytitle2, Waves_Info)
    
    return Waves_Info

In [9]:
def Get_Correlations(State_Code, list_of_dfs, Waves_Info):
    report_date_list = list_of_dfs[6]
    fmt0 = '%y/%m/%d'
    start_date = datetime.strptime(report_date_list[0], fmt0)
    end_date = datetime.strptime(report_date_list[-1], fmt0)
    fmt = '%b %d %Y'
    
    Waves_Dict = {}
    for key in Waves_Info.keys():
        try:
            Waves_Dict[key] = ('{}'.format((Waves_Info[key]['start'][1]).strftime(fmt)), 
                               '{}'.format((Waves_Info[key]['end'][1]).strftime(fmt)))
        except AttributeError:
            if Waves_Info[key]['end'][1] == 'NA':
                Waves_Dict[key] = ('{}'.format((Waves_Info[key]['start'][1]).strftime(fmt)), 
                                   end_date.strftime(fmt))
    Waves_Dict['Overall'] = (start_date.strftime(fmt), 
                             end_date.strftime(fmt))
    
    Scope_W = widgets.RadioButtons(options = ['County', 'Congressional District'],
                                   description = 'Scope',
                                   style = {'description_width': '38px'}, 
#                                    layout = {'width': '200px'},
                                   disabled = False)
    
    R_W = widgets.RadioButtons(options = ['R', 'R squared'], 
                               description = '', 
                               disabled = False)
    
    Rmethod_W = widgets.RadioButtons(options = ['pearson', 'spearman'], 
                                     description = '', 
                                     disabled = False)
    
    MCorr_W = widgets.Checkbox(value = True, 
                               description = 'Correlation Matrix', 
                               indent = False)
    Mscatter_W = widgets.Checkbox(value = False, 
                                  description = 'Scatter Matrix', 
                                  indent = False)
    Partial_Corr_W = widgets.Checkbox(value = False, 
                                      description = 'Partial Correlation Matrix', 
                                      indent = False)
    
    variables_data = ['COVID Outcome', 'Population', 'Age', 'Race', 'Income', 'Politics', 'Vaccination Status']
    
    List_Keys_Dict = {('Population', 'County'): 0, ('Population', 'Congressional District'): 1, 
                      ('Age', 'County'): 2, ('Age', 'Congressional District'): 3, 
                      ('Race', 'County'): 2, ('Race', 'Congressional District'): 3, 
                      ('Income', 'County'): 2, ('Income', 'Congressional District'): 3, 
                      ('Politics', 'County'): 2, ('Politics', 'Congressional District'): 3, 
                      ('COVID Outcome', 'County'): 4, ('COVID Outcome', 'Congressional District'): 5, 
                      ('Vaccination Status', 'County'): 10, ('Vaccination Status', 'Congressional District'): 11}
    
    Measure_Options = {('COVID Outcome', False): [], ('COVID Outcome', True): ['Deaths per population', 
                                                                               'Hospitalizations per population', 
                                                                               'Cases per population', 
                                                                               'Deaths per Hospitalizations'],
                       ('Population', False): [], ('Population', True): ['Population Density', 'Population'],
                       ('Age', False): [], ('Age', True): ['Percent 65 years and over', 'Percent Under 45 years'], 
                       ('Race', False): [], ('Race', True): ['Percent White, Not Hispanic', 
                                                             'Percent Black or African American, Not Hispanic', 
                                                             'Percent Hispanic or Latino'],
                       ('Income', False): [], ('Income', True): ['Median Income', 'Per Capita Income', 
                                                                 'Median Household Income'],
                       ('Politics', False): [], ('Politics', True): ['PVI_R', 'PVI_D'], 
                       ('Vaccination Status', False): [], ('Vaccination Status', True): ['TBD']}
    Wave_Options = {True: Waves_Dict.keys(), 
                    False: []}
    
    labels_data = ['Variables', 'Measure', 'Wave']
    
    Variables_W = [widgets.Checkbox(value = True, 
                                    description = label, 
                                    indent = False) 
                   for label in variables_data]
    
    index1 = variables_data.index('COVID Outcome')
    index2 = variables_data.index('Vaccination Status')
    Variables_W[index2].value = False
    
    Labels_W = [widgets.Label(value = label, layout = {'height': '30px'}) 
                for label in labels_data]
    
    Measures_W = [widgets.Dropdown(options = Measure_Options[(Variables_W[i].description, Variables_W[i].value)], 
                                   disabled = False) 
                  for i in range(len(Variables_W))]
        
    Wave_W = widgets.Dropdown(options = Wave_Options[Variables_W[index1].value | Variables_W[index2].value], 
                              layout = {'width': '100px'}, 
                              description = '', 
                              disabled = False)
    
    try:
        SLabel_W = widgets.Label(value = Waves_Dict[Wave_W.value][0], 
                                 layout = {'width': '150px', 'height': '22px'})
        ELabel_W = widgets.Label(value = 'through  ' + Waves_Dict[Wave_W.value][1], 
                                 layout = {'width': '150px', 'height': '22px'})
    except KeyError:
        SLabel_W = widgets.Label(value = '', layout = {'width': '150px', 'height': '22px'})
        ELabel_W = widgets.Label(value = '', layout = {'width': '150px', 'height': '22px'})
    
    def update_measures(*args):
        for i in range(len(Measures_W)):
            Measures_W[i].options = Measure_Options[(Variables_W[i].description, Variables_W[i].value)]
    
    for i in range(len(Variables_W)):
        Variables_W[i].observe(update_measures, 'value')
        
    def update_wave(*args):
        Wave_W.options =  Wave_Options[Variables_W[index1].value | Variables_W[index2].value]
        
    Variables_W[index1].observe(update_wave, 'value')
    Variables_W[index2].observe(update_wave, 'value')
    
    def update_labels(*args):
        try:
            SLabel_W.value = Waves_Dict[Wave_W.value][0]
            ELabel_W.value = 'through  ' + Waves_Dict[Wave_W.value][1]
        except KeyError:
            SLabel_W.value = ''
            ELabel_W.value = ''
            
    Wave_W.observe(update_labels, 'value')
    
# =================================================================================================================
    def Display_Matrices(scope, variables, measures, R, Rmethod, MCorr, Mscatter, Partial_Corr):        
        var_lists = {}
        for i in range(len(variables)):
            v = variables[i]
            m = measures[i]
            if v.value:
                k = List_Keys_Dict[(v.description, scope)]
                x = m.value
                if k == 3:
                    x = x.replace('PVI', 'PVI_2022')
                if k not in var_lists:
                    var_lists[k] = [x]
                else:
                    var_lists[k].append(x)
        
        if len(var_lists) < 2:
            return
    
        num_vars = 0
        for k in var_lists.keys():
            df = list_of_dfs[k].copy()
            if k < 4:
                df.drop(df.columns.difference(var_lists[k]), axis = 1, inplace = True)
            elif (k == 4) or (k == 5):
                sIndex = {4: 'County Code', 5: 'CDistrict'}
                df.set_index([sIndex[k], 'Report Date'], inplace = True)
                df.sort_index(inplace = True)
                df.drop(df.columns.difference(['Total Cases', 'Deaths', 'Hospitalizations']), axis = 1, inplace = True)
                df.rename(columns={"Total Cases": "Cases"}, inplace = True)
                wave = Wave_W.value
                
                if wave == 'Wave 1':
                    end_date = Waves_Info[wave]['end'][1].strftime(fmt0)
                    df = df.loc[(slice(None), end_date), :]
                elif wave == 'Overall':
                    end_date = report_date_list[-1]
                    df = df.loc[(slice(None), end_date), :]
                else:
                    n = int(wave[-1])
                    pre_wave = 'Wave ' + str(n - 1)
                    next_wave = 'Wave ' + str(n + 1)
                    if next_wave in Waves_Info.keys():
                        end_date = Waves_Info[wave]['end'][1].strftime(fmt0)
                    else:
                        try:
                            end_date = Waves_Info[wave]['end'][1].strftime(fmt0)
                        except AttributeError:
                            end_date = report_date_list[-1]
                    start_date = Waves_Info[pre_wave]['end'][1].strftime(fmt0)
                    df = df.loc[(slice(None), [start_date, end_date]), :]
                    df.loc[(slice(None), start_date), :] *= -1
                df = df.groupby(level = 0).sum()
                var = var_lists[k][0]
                if ' per population' in var:
                    col = var.replace(' per population', '')
                    new_col = col + '/1000 pop' + '(' + wave + ')'
                    df[new_col] = Get_Per_Pop(State_Code, scope, df, col)*1000
                elif var == 'Deaths per Hospitalizations':
                    new_col = var + '(' + wave + ')'
                    df[new_col] = (df['Deaths'].divide(df['Hospitalizations'])).replace(np.nan, 0)
                else:
                    new_col = var + '(' + wave + ')'
                    df[new_col] = df[var]
                df.drop(df.columns.difference([new_col]), axis = 1, inplace = True)
            else:
                continue
                
            if num_vars == 0:
                Matrix = df.copy()
                num_vars += 1
            else:
                Matrix = pd.concat([Matrix, df], axis = 1)
                num_vars += 1
         
        if num_vars == 0:
            return
        
        Matrix.columns = Matrix.columns.str.replace('Percent', '%').str.replace(', Not Hispanic', '')
        
        if MCorr:
            Mcorr_HeatMap(Matrix, Rmethod, R)
            
        if Mscatter:
            Mcorr_Scatter(Matrix, Rmethod)
            
        if Partial_Corr:
            if Rmethod == 'spearman':
                print('only Pearson method is used for pairwise correlations given all others')
            Mp_corr_HeatMap(Matrix, R)

        return 
# =================================================================================================================

    widget = widgets.interactive(Display_Matrices, {'manual' : True, 'manual_name' : 'Run'}, 
                                 scope = Scope_W, variables = [Variables_W], measures = [Measures_W], 
                                 R = R_W, Rmethod = Rmethod_W, MCorr = MCorr_W, 
                                 Mscatter = Mscatter_W, Partial_Corr = Partial_Corr_W)
    
    col1_2 = widgets.VBox(children = [MCorr_W, Mscatter_W, Partial_Corr_W])
    col1_3 = widgets.VBox(children = [R_W, Rmethod_W])
    row1 = widgets.HBox([Scope_W, col1_2, col1_3])
    
    row2 = widgets.HBox(children = Labels_W, 
                        layout = widgets.Layout(display="flex", justify_content = 'space-between', width = '85%'))
    
    col1 = widgets.VBox(children = Variables_W)
    col2 = widgets.VBox(children = Measures_W)
    col3 = widgets.VBox(children = [Wave_W, SLabel_W, ELabel_W])
    
    row3 = widgets.HBox([col1, col2, col3], 
                        layout = widgets.Layout(display="flex", justify_content = 'space-between', width = '90%'))
    
    output = widget.children[-1]
    run_button = widget.children[-2]
    
    display(widgets.VBox([row1, row2, row3, run_button, output], 
                         layout = widgets.Layout(display="flex", justify_content="flex-start")))
    
    return